In [1]:
import math
import numpy as np

In [2]:

#eq_1 = np.random.uniform(low=0, high=13.3, size=(2,)).tolist()
#eq_2 =  np.random.uniform(low=0, high=13.3, size=(2,)).tolist()
#A = np.array([eq_1, eq_2])
#b = np.random.uniform(low=0.5, high=13.3, size=(2,)).tolist()
#b = np.array(b)
#b = b.T

#coefficients =np.array([[4.03,2.16],[6.21,3.35]])
#res=np.array([-4.61,-7.19])
#res=res.T

In [3]:
def roundoff(number, significant_digits):
    d, n = math.modf(number)
    count=0
    num = n
    while(num > 0):
        num = num // 10
        count = count + 1
    if n == 0:
        significant_digits = significant_digits-1
    d = round(d, significant_digits-count)
    return n+d

In [4]:
def roundoff_(number, significant_digits):
    number = number + (5 * (10**(-(significant_digits+1))))
    stepper = 10.0 ** significant_digits
    number = math.trunc(stepper * number) / stepper
    return number

In [5]:
def np_roundoff_(numbers , significant_digits):
    i=0
    while(i < len(numbers)):
        numbers[i] = numbers[i] + (5 * (10**(-(significant_digits+1))))
        stepper = 10.0 ** significant_digits
        numbers[i] = math.trunc(stepper * numbers[i]) / stepper
        i =i+1
    return numbers

In [15]:
def np_roundoff(numbers, significant_digits):
    i=0
    while(i < len(numbers)):
        d, n = math.modf(numbers[i])
        count=0
        num = n
        while(num > 0):
            num = num // 10
            count = count + 1
        if n == 0:
            significant_digits = significant_digits-1
        d = round(d, significant_digits-count)
        numbers[i] = n+d
        i=i+1
    return numbers

In [8]:
def GENP(A, b):
    '''
    Gaussian elimination with no pivoting.
    % input: A is an n x n nonsingular matrix
    %        b is an n x 1 vector
    % output: x is the solution of Ax=b.
    % post-condition: A and b have been modified. 
    '''
    n =  len(A)
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between A & b.", b.size, n)
    for pivot_row in range(n-1):
        for row in range(pivot_row+1, n):
            multiplier = A[row][pivot_row]/A[pivot_row][pivot_row]
            multiplier = r(multiplier, sd) # roundoff
            #the only one in this column since the rest are zero
            A[row][pivot_row] = multiplier
            for col in range(pivot_row + 1, n):
                A[row][col] = A[row][col] - multiplier*A[pivot_row][col]
                A[row][col]=r(A[row][col],sd) # roundoff
                
            #Equation solution column
            b[row] = b[row] - multiplier*b[pivot_row]
            b[row] = r(b[row],sd) #roundoff
    x = np.zeros(n)
    k = n-1
    x[k] = b[k]/A[k,k]
    x[k] = r(x[k],sd)
    while k >= 0:
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
        x[k] = r(x[k],sd)
        k = k-1
    return x

In [9]:
def GEPP(A, b, doPricing = True):
    '''
    Gaussian elimination with partial pivoting.
    
    input: A is an n x n numpy matrix
           b is an n x 1 numpy array
    output: x is the solution of Ax=b 
            with the entries permuted in 
            accordance with the pivoting 
            done by the algorithm
    post-condition: A and b have been modified.
    '''
    n = len(A)
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between"+
                         "A & b.", b.size, n)
    # k represents the current pivot row. Since GE traverses the matrix in the 
    # upper right triangle, we also use k for indicating the k-th diagonal 
    # column index.
    
    # Elimination
    for k in range(n-1):
        if doPricing:
            # Pivot
            maxindex = abs(A[k:,k]).argmax() + k
            if A[maxindex, k] == 0:
                raise ValueError("Matrix is singular.")
            # Swap
            if maxindex != k:
                A[[k,maxindex]] = A[[maxindex, k]]
                b[[k,maxindex]] = b[[maxindex, k]]
        else:
            if A[k, k] == 0:
                raise ValueError("Pivot element is zero. Try setting doPricing to True.")
        #Eliminate
        for row in range(k+1, n):
            multiplier = A[row,k]/A[k,k]
            multiplier = r(multiplier,sd) # roundoff
            A[row, k:] = A[row, k:] - multiplier*A[k, k:]
            A[row,k:] = nr(A[row,k:],sd) # roundoff
            b[row] = b[row] - multiplier*b[k]
            b[row] = r(b[row],sd) # roundoff
    # Back Substitution
    x = np.zeros(n)
    for k in range(n-1, -1, -1):
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
        x[k] = r(x[k], sd) # roundoff
    return x



In [33]:
sds=[3,4,5]

eq_1 = np.random.uniform(low=1.3, high=10.3, size=(2,)).tolist()
eq_2 =  np.random.uniform(low=1.3, high=10.3, size=(2,)).tolist()
coeff = np.array([eq_1, eq_2])
rhs = np.random.uniform(low=1.0, high=10.3, size=(2,)).tolist()
rhs = np.array(rhs)
rhs = rhs.T

r = roundoff_
nr = np_roundoff_

In [36]:
print('Gauss elimination without pivoting ...')
print('coeffients - ', coeff)
print()
print('rhs -', rhs)
print()
for sig_digit in sds:
    sd = sig_digit
    res = GENP(np.copy(coeff), np.copy(rhs))
    print(f'significant_digit= {sd}, X1 = {res[0]}, X2= {res[1]}')

Gauss elimination without pivoting ...
coeffients -  [[2.47716403 3.28299248]
 [4.78046469 8.97875939]]

rhs - [2.93357595 3.420466  ]

significant_digit= 3, X1 = 2.307, X2= -0.847
significant_digit= 4, X1 = 2.3076, X2= -0.8476
significant_digit= 5, X1 = 2.30776, X2= -0.84774


In [37]:
print('Gauss elimination without pivoting ...')
print('coeffients - ', coeff)
print()
print('rhs -', rhs)
print()
for sig_digit in sds:
    sd = sig_digit
    res = GEPP(np.copy(coeff), np.copy(rhs))
    print(f'significant_digit= {sd}, X1 = {res[0]}, X2= {res[1]}')

Gauss elimination without pivoting ...
coeffients -  [[2.47716403 3.28299248]
 [4.78046469 8.97875939]]

rhs - [2.93357595 3.420466  ]

significant_digit= 3, X1 = 2.31, X2= -0.849
significant_digit= 4, X1 = 2.3075, X2= -0.8476
significant_digit= 5, X1 = 2.30784, X2= -0.84779
